In [1]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import glob

In [2]:
sia = SentimentIntensityAnalyzer()

In [3]:
filenames = glob.glob('reddit-subset2\submissions\part-*.json.gz')
ind_df = (pd.read_json(f, lines=True) for f in filenames)
df = pd.concat(ind_df, ignore_index=True)
df = df[(df['selftext'] != '[removed]') & (df['selftext'] != '[deleted]') & (df['subreddit'] != 'Quebec')]

In [4]:
def get_score(input):
    scores = sia.polarity_scores(input)
    return scores['compound']

In [5]:
get_scores = np.vectorize(get_score)

In [6]:
df['score'] = get_scores(df['selftext'])

In [7]:
sentiment_score_annual = df.groupby(['subreddit', 'year']).agg({'score': 'mean'})

In [8]:
sentiment_score_annual.to_csv('sentiment_score_annual.csv')

In [10]:
sentiment_score_quarterly = df

In [11]:
sentiment_score_quarterly['date'] = pd.to_datetime(df['year'].astype(str) + df['month'].astype(str), format='%Y%m')

In [13]:
sentiment_score_quarterly.groupby(['subreddit', sentiment_score_quarterly['date'].dt.to_period('Q')]).agg({'score': 'mean'})

score
subreddit    date            
Manitoba     2016Q1  0.525941
             2016Q2  0.255661
             2016Q3  0.538294
             2016Q4  0.538191
             2017Q1  0.451412
...                       ...
saskatchewan 2022Q1  0.288574
             2022Q2  0.310727
             2022Q3  0.275282
             2022Q4  0.287658
             2023Q1  0.258810

[348 rows x 1 columns]

In [14]:
sentiment_score_quarterly.to_csv('sentiment_score_quarterly.csv')